## **2. Multi Layer Perceptron**

In [ ]:
import math
import random
import torch
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
words = open('./data/names.txt', 'r').read().splitlines()
words[:10]

In [ ]:
# Vogliamo ora predire il carattere successivo data una certa sequenza di caratteri
# vocabolario di caratteri e mapping a interi
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [ ]:
# costruiamo il dataset
block_size = 3 # numero di caratteri presi in considerazione per predirre il carattere successivo
X, Y = [], []
for w in words:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '-->', itos[ix])
        context = context[1:] + [ix] # togli il primo carattere e aggiugi l'ultima y
X = torch.tensor(X)
Y = torch.tensor(Y)
# otteniamo dati in input di dimensione n°esempi x dimensione blocco e vettore di risposta di dimensione n°esempi

In [ ]:
# Strato iniziale -> prendo l'input X e lo trasformo tramite tabella di look-up
C = torch.randn((27, 2)) # tabella di look-up, dove per ogni possibile carattere (in totale sono 27), viene assegnato un valore inizialmente randomico
emb = C[X] # otteniamo i valori embedding corrispondenti ai valori nella lista X (per ogni valore di X che è composto da 3 caratteri, ottengo i 3 valori della look-up table corrispondenti, dove ognuno di essi è composto da 2 valori)
emb.shape

In [ ]:
# Stato intermedio -> il numero di neuroni è 100
W1 = torch.randn((6, 100)) # rappresenta i pesi dello stato intermedio, dato che ogni embedding è formato da 3 coppie, ho 6 input per ogni neurone e quindi 6 pesi. 
b1 = torch.randn(100)

# in questo straot moltiplico ogni embedding per il peso, aggiungo il bias e applico la funzione di attivazione,
# quindi calcolo emb @ W1 + b1 e applico una funzione di attivazione come tanh

# per moltiplicare gli embedding con i pesi dobbiamo appiattire gli embedding a bi-dimensionali così da ottenere (n x 6) @ (6 X 100) = (n x 100)
# torch.cat(torch.unbind(emb, 1), 1)
print(emb.view(emb.shape[0],6).shape) # metodo migliore per appiattire dimensionalità, costrutto logico per rappresentare la memoria fisica

h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1) # funzione di attivazione su W * X + b
print(h)
print(h.shape)

In [ ]:
# Strato finale
W2 = torch.randn((100,27)) # input è il numero di neuroni nello strato intermedio, l'output è la probabilità per ciascun carattere
b2 = torch.randn((27))

logits = h @ W2 + b2 # input * pesi + bias
# Softmax come funzione di attivazione 
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
print(probs.shape)

In [ ]:
# Quello che vogliamo fare ora è vedere ogni la probabilità di ottenere la predizione corretta per ogni riga e tramite la loss function aggiustare questa misura iterativamente
probs[torch.arange(X.shape[0]), Y] # probabilità di ottenere la predizione corretta
loss = -probs[torch.arange(X.shape[0]), Y].log().mean()
# possiamo calcolare la loss in maniera più diretta tramite -> loss = F.cross_entropy(logits, Y)
loss

In [ ]:
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

for _ in range(10):
    # Forward pass:
    emb = C[X]
    h = torch.tanh(emb.view(X.shape[0], 6) @ W1 + b1)
    logits = h@ W2 + b2
    loss = F.cross_entropy(logits, Y)
    print(loss.item())
    # Backward pass:
    for p in parameters:
        p.grad = None
    loss.backward()
    # Update:
    for p in parameters:
        p.data += -0.1 * p.grad

In [ ]:
# Nella realtà vengono eseguiti i passi di forward e backward non su tutto il dataset ogni volta ma su un campione randomico di esso chiamato minibatch, in questo modo il processo è più rapido

for _ in range(10000):
    ix = torch.randint(0, X.shape[0], (32,)) # selezione randomicamente 32 righe rispetto a quelle possibili
    # Forward pass:
    emb = C[X[ix]] # (32, 3, 2)
    h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1) # (32, 100)
    logits = h @ W2 + b2 # (32, 27)
    loss = F.cross_entropy(logits, Y[ix])
    # Backward pass:
    for p in parameters:
        p.grad = None
    loss.backward()
    # Update:
    for p in parameters:
        p.data += -0.1 * p.grad
print(loss.item())

# dato che stiamo eseguendo dei minibatches, la qualità del gradiente è minore, quindi la direzione non è affidabile come nel caso del dataset completo
# ma è molto meglio avere un gradiente approssimativo e fare molti più step rispetto a valutare il gradiente esatto e farne pochi.

In [ ]:
# possiamo ora valutare la loss function sul dataset completo e vedere come è andata:
emb = C[X] # (32, 3, 2)
h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Y)
loss

In [ ]:
# Vediamo che otteniamo una loss sempre minore, questo ci porterebbe a dire che abbiamo allenato il modello correttamente ed esso è ora in grado di fare previsioni più precise
# Questo però non è sempre vero, infatti potrebbe essere che il modello stia overfittando sui dati -> No Buono !
# Questo accade solitamente quanto si ha un numero di parametri elevato, i quali sono capaci di imparare tutte le casistiche presenti nel dataset. In questo modo il modello non sta imparando i pattern ma semplicemente imparando a memoria le combinazioni del dataset ottenendo semplicemente gli stessi valori senza creare nulla di nuovo

# Per evitare problemi di overfit, lo standard è dividere il dataset in 3 partizioni: training set, validation set e test set 
# - training -> utilizzato per otimizzare i parametri del modello
# - validation -> tuning degli hyper-parametri del modello: nel nostro caso potrebbero essere la dimensione dello strato intermedio o la dimensione dell'embedding table
# - test -> valuta le performance del modello alla fine

def build_dataset(words, n):
    block_size = n
    X, Y = [], []
    for w in words:
        #print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '-->', itos[ix])
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

X_train, y_train = build_dataset(words[:n1],3)
X_valid, y_valid = build_dataset(words[n1:n2],3)
X_test, y_test = build_dataset(words[n2:],3)

In [ ]:
C = torch.randn((27, 2))
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
W2 = torch.randn((100,27))
b2 = torch.randn((27))
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

In [ ]:
# Training:
for _ in range(20000):
    ix = torch.randint(0, X_train.shape[0], (32,)) # selezione randomicamente 32 righe rispetto a quelle possibili
    # Forward pass:
    emb = C[X_train[ix]] # (32, 3, 2)
    h = torch.tanh(emb.view(emb.shape[0], 30) @ W1 + b1) # (32, 100)
    logits = h @ W2 + b2 # (32, 27)
    loss = F.cross_entropy(logits, y_train[ix])
    # Backward pass:
    for p in parameters:
        p.grad = None
    loss.backward()
    # Update:
    for p in parameters:
        p.data += -0.01 * p.grad
print(loss.item())

In [ ]:
emb = C[X_valid]
h = torch.tanh(emb.view(emb.shape[0], 30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, y_valid)
loss
# Possiamo vedere che per ora stiamo underfittando, in quanto la loss del training e del validation set sono molto simili e quindi il modello non è abbastanza potente per semplicemente memorizzare i dati
# Possiamo quindi aspettarci dei miglioramenti se aumentiamo la dimensione della rete

In [ ]:
# aumento il numero di neuroni nell'hidden layer e la dimensione della tabella di look-up
C = torch.randn((27, 10))
W1 = torch.randn((30, 200))
b1 = torch.randn(200)
W2 = torch.randn((200,27))
b2 = torch.randn((27))
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True
# e rieseguiamo il codice nei 2 blocchi superiori
# quello che stiamo facendo sono degli esperimenti nel vedere come possiamo aggiustare i parametri in modo da ottenere performance migliori (non è una scienza esatta)

In [ ]:
# Proviamo ora a generare nomi dal modello:
for i in range(10):
    out = []
    context = [0] * 3
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(emb.shape[0], 30) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, replacement=True).item() # campionamente per ottenere successore
        out.append(ix)
        context = context[1:] + [ix]
        if (ix==0): # se è 0 vuol dire che la parole è terminata
            break

    print(''.join(itos[i] for i in out))